## Ground Segment Scheduling Optimization

This notebook demonstrates an implementation of a custom contact scheduling algorithm and the integration of this
algorithm into a Sedaro simulation via cosimulation. We use a `GroundSegment` to model the [Atlas Enterprise ground
station network](https://atlasspace.com/wp-content/uploads/2023/07/07192023-ANTENNA-NETWORK-ENTERPRISE-SITES.pdf), and
we simulate a scenario of a Walker delta 60: 24/3/3 constellation with uplink and downlink requirements. The scheduling
is posed as an integer linear programming problem which maximizes downlinked data under a minimum uplink constraint [1].

You can view the results of this cosimulation in `sim_results.ipynb`, and you can fork the demo scenario to run the
cosimulation yourself (since you have read permission but not write/simulate). Make sure to update `scenario_branch` in
"Setup" if you do so.

[1]Eddy, D., Ho, M., and Kochenderfer, M. J., “Optimal Ground Station Selection for Low-Earth Orbiting Satellites”, 
        arXiv e-prints, Art. no. arXiv:2410.16282, 2024. doi:10.48550/arXiv.2410.16282.

This notebook requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the root directory of this repository (two levels above this notebook) with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

### Setup

Run the cell below once to install required packages


In [1]:
%pip install -r requirements.txt

  Using cached attrs-24.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
  Using cached charset_normalizer-3.4.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (34 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached cloudpickle-3.1.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached dask-2024.12.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached dask_expr-1.1.20-py3-none-any.whl.metadata (2.6 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached fastjsonschema-2.21.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached flatdict-4.0.1-py3-none-any.whl
  Using cached frozendict-2.3.10-py3-none-any.whl.metadata (20 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached grpcio-1.68.1-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.9 kB)
  Using cached grpcio_tools-1.68.1-cp312-cp312-macosx_10_9_universal2.whl.metadata (5.3 kB)
  Using cached idna

Import modules and configure some settings for the cosimulation. You can change the minimum uplink requirement here or
change the branch ids to use this notebook for your own repository.

In [1]:
from scipy.optimize import milp, LinearConstraint, Bounds
import numpy as np
from utils import sedaroLogin, contact_booleans_to_intervals, selected_contacts_to_schedule
from typing import Any
import time
from math import ceil

# Settings
scenario_branch = 'PPPsqqNVh7HjbZsB7vtT7d'
ground_segment_template = 'PPPsg2D3KJwJMZ6mfPhHgb' # Read-only
gs_agent_name = 'Atlas (Enterprise)'
minimum_uplink = 1000000000 # bits
schedule_period = 4 # hours


# Constants
RESOLUTION_SECONDS = 10. # Sedaro provides a handle for scheduling contacts at 10s resolution

client = sedaroLogin()
scenario = client.scenario(scenario_branch)
template = client.agent_template(ground_segment_template)
uplink_bitrate = template.ScheduledTransmitInterface.get_first().onBitRate


Set up scheduling as an Integer Linear Programming Problem

In [2]:
def optimize_schedule(
        contact_intervals: list[tuple[int, int]],
        c_location: dict[str, list[int]],
        c_tg: dict[str, list[int]],
        durations: list[int],
) -> tuple[dict[str, Any], ...]:
    '''
    Based on reference [1], set up the contact scheduling as an integer linear programming problem, then solve using
    scipy.optimize.milp.

    Args:
        contact_intervals: A list of all contacts, represented as a tuple of start and stop indices
        c_location: A dictionary of the list of contact indices for each location
        c_tg: A dictionary of the list of contact indices for each target group
        durations: The duration of each contact in contact_intervals

    [1] Eddy, D., Ho, M., and Kochenderfer, M. J., “Optimal Ground Station Selection for Low-Earth Orbiting Satellites”, 
        arXiv e-prints, Art. no. arXiv:2410.16282, 2024. doi:10.48550/arXiv.2410.16282.
    '''
    n_contacts = len(contact_intervals)
    # The solution vector x is a list of binaries of length 2*len(C). The first len(C) selects contacts for downlink
    # and the second len(C) selects contacts for uplink. We will maximize data downlink while satisfying the minimum 
    # uplink requirements. 

    # First, well set up the objective function. The amount of data linked down is proportional to link duration
    f = -np.hstack([np.array(durations), np.zeros(n_contacts)])

    # Next, we set up the constraints.
    constraints = []
    # The first constraint is the aforementioned minimum uplink requirement per plane
    minimum_uplink_steps = minimum_uplink / uplink_bitrate / RESOLUTION_SECONDS
    for indices in c_tg.values():
        # element-wise product to select only the contacts that are in the target group
        plane_weight = np.array(durations) * np.array([1 if i in indices else 0 for i in range(n_contacts)])
        A = np.hstack([np.zeros(n_contacts), plane_weight])
        constraints.append(LinearConstraint(A, lb=minimum_uplink_steps))

    # The second is that each antenna can only communicate with one spacecraft at a time, based on eqn. 11
    # The third is like it. Eddy et al. require each spacecraft only communicate with one antenna at a time (eqn. 12),
    # but we require the stronger condition that each target group communicate with one antenna at a time. These are 
    # similar constraints, so we set up a helper function to create all of these constraints efficiently.
    def exclusion_matrix(entity_contacts):
        '''
        Returns a matrix for evaluating exclusion constraints. Each row will find the sum of conflicting contacts. This
        matrix can be used to create an efficient LinearConstraint to enforce A@x <= 1.
        '''
        exclusion_rows = []
        for contact_indices in entity_contacts.values():
            # Contacts sorted by start time. List of pairs where the first element is the start and stop time and the
            # second is the index in contact_indices pre-sorting
            flattened_schedule_indices = sorted([(contact_intervals[i], i) for i in contact_indices], key=lambda x: x[0][0])
            # Transpose
            flat_schedule, sorted_indices = zip(*flattened_schedule_indices)
            # Loop through the flattened schedule to look for any overlapping contacts
            for i in range(1, len(flat_schedule)):
                if flat_schedule[i][0] < flat_schedule[i-1][1]:
                    # These contacts overlap, so we must make a constraint that only one of them is selected, including uplink
                    contact1_index = sorted_indices[i]
                    contact2_index = sorted_indices[i-1]
                    A = np.zeros(2*n_contacts)
                    A[contact1_index] = 1
                    A[contact2_index] = 1
                    A[contact1_index+n_contacts] = 1
                    A[contact2_index+n_contacts] = 1
                    exclusion_rows.append(A)
        return np.vstack(exclusion_rows) if exclusion_rows else None
    
    # Add constraint if any location overlaps exist
    if (station_exclusion_rows := exclusion_matrix(c_location)) is not None:
        constraints.append(LinearConstraint(station_exclusion_rows, ub=1))
    # Same for target groups
    if (tg_exclusion_rows := exclusion_matrix(c_tg)) is not None:
        constraints.append(LinearConstraint(tg_exclusion_rows, ub=1))

    # Run the solver and time it for fun
    start = time.time()
    result = milp(
        f, # f@x is minimized
        integrality=np.ones(2*n_contacts), # all params are integers
        bounds=Bounds(0, 1), # all params are binary
        constraints=constraints
        ) 
    end = time.time()
    print(f"Optimization took {end-start} seconds. {result.message}")

    return result.x
    

Set up cosimulated external state. You only need to run this if you change the scheduler id or require any additional
schedule inputs. (However, the old external state is cleared, so it will not hurt to run if you're not sure.)

In [3]:
# clear out existing blocks
scenario.delete_all_external_state_blocks()

# Get the ground segment agent
gs_agent = scenario.Agent.get_where(name=gs_agent_name)[0]

# The existing scheduler will provide us a handle for the the contacts, and we can overwrite the schedule
scheduler = template.ContactScheduler.get_first()
external_state = scenario.PerRoundExternalState.create(
    consumed = f'''(
        block!("{scheduler.id}").(projectedContacts, targetSeries),
        block!("{scheduler.id}").interfaces.(id, type),
        block!("{scheduler.id}").interfaces.linkTargetGroup.(id, targets.id, targets.agentId),
        time
    )''',
    produced = f'(block!("{scheduler.id}").schedule,)',
    engine = 'cdh',
    agents = [gs_agent.id]
)
ancillary_state = scenario.PerRoundExternalState.create(
    consumed='(elapsedTime as Duration.hour,)',
    produced = f'''(
        block!("{scheduler.id}")._generateNewSchedule,
        block!("{scheduler.id}").cycleDuration as Duration.hour,
    )''',
    engine = 'cdh',
    agents = [gs_agent.id]
)

## Cosimulate

Once we start the simulation, we'll check the elapsed sim time to see if a new schedule is needed. If so, we'll get the
required inputs to determine the contact intervals and pass them into the optimizer. The optimizer will choose the
contacts, which are then parsed into a schedule and sent to the simulation.

The Exception at the end of the simulation is expected and does not necessarily mean that the cosimulation failed.

In [4]:
schedules_published = 0
schedule = None

# Start the simulation
with scenario.simulation.start(wait=True) as simulation_handle:
    print('Simulation started!')
    async with simulation_handle.async_channel() as channel:
        print('Channel opened!')
        # Simulation loop
        while True:
            ancillary_data = await channel.consume(agent_id=gs_agent.id, external_state_id=ancillary_state.id)
            # We don't know if we need the detailed schedule inputs yet, so we keep this as a future
            schedule_inputs = channel.consume(agent_id=gs_agent.id, external_state_id=external_state.id)
            t_h = ancillary_data[0]
            # Check if the schedule period has elapsed
            if ceil(t_h / schedule_period) > schedules_published:
                print("Generating new schedule!")
                schedules_published += 1
                # Set _generateNewSchedule to True to trigger contact projection
                await channel.produce(agent_id=gs_agent.id, external_state_id=ancillary_state.id,values=(True, schedule_period))
                print("Getting inputs...")
                (projected_contacts, target_series), interface_ids, tg_targets, mjd = await schedule_inputs
                print('Setting up optimizer...')
                # Pre-processing to projected contacts as discrete intervals
                c_t, contact_intervals, c_location, c_satellite, c_tg, durations, contact_labels = contact_booleans_to_intervals(projected_contacts, tg_targets)
                # This runs the optimizer which selects contacts
                contacts_selected = optimize_schedule(contact_intervals, c_location, c_tg, durations)
                # Parse the optimizer output into a schedule that the sim can interpret
                print('Parsing optimizer output...')
                schedule = selected_contacts_to_schedule(
                    contacts_selected, 
                    contact_intervals,
                    contact_labels,
                    target_series,
                    interface_ids,
                    tg_targets,
                    mjd,
                    RESOLUTION_SECONDS,
                    )
                await channel.produce(agent_id=gs_agent.id, external_state_id=external_state.id, values=(schedule,))
                print('Done! Simulation is executing the schedule.')
            else:
                await channel.produce(agent_id=gs_agent.id, external_state_id=ancillary_state.id, values=(False, schedule_period))
                await channel.produce(agent_id=gs_agent.id, external_state_id=external_state.id, values=(schedule,))
                await schedule_inputs

Simulation started!
Channel opened!
Generating new schedule!
Getting inputs...
Setting up optimizer...
Optimization took 0.32497096061706543 seconds. Optimization terminated successfully. (HiGHS Status 7: Optimal)
Parsing optimizer output...
Done! Simulation is executing the schedule.
Generating new schedule!
Getting inputs...
Setting up optimizer...
Optimization took 9.573915958404541 seconds. Optimization terminated successfully. (HiGHS Status 7: Optimal)
Parsing optimizer output...
Done! Simulation is executing the schedule.
Generating new schedule!
Getting inputs...
Setting up optimizer...
Optimization took 0.1282179355621338 seconds. Optimization terminated successfully. (HiGHS Status 7: Optimal)
Parsing optimizer output...
Done! Simulation is executing the schedule.


ERROR:root:Unexpected response state: 1
ERROR:root:Consume operation failed for index 8641: Unexpected response state.
ERROR:root:Cosimulation session encountered an error: Unexpected response state.


Exception: Unexpected response state.

## Plot resulting schedule

In [5]:
from ipynb.fs.defs.sim_results import make_contact_schedule_plots

pure_sim = scenario.simulation.results()
ground_segment_results = pure_sim.agent(gs_agent_name)
make_contact_schedule_plots(template, ground_segment_results)

Downloading...
...download complete!
